In [16]:
## SIMULATION PARAMETERS
verbosity         = 8        # Set debug print statement verbosity level (0 = Standard, -1 = Off)
use_mass_units    = True     # Toggle whether calculations / results are given in units of pi-axion mass (True) or eV (False)
use_natural_units = True     # Toggle whether calculations / results are given in c = h = G = 1 (True) or SI units (False)   || NOTE: full SI/phsyical unit support is still WIP!!
save_output_files = True     # Toggle whether or not the results from this notebook run are written to a data directory

## CONFIG PARAMETERS
config_name = 'piaxi_debug'   # Descriptive name for the given parameter case. Output files will be saved in a directory with this name.
seed = 269934808235809364794393739371438879116                      # rng_seed, integer value (None for random)
#seed = 178086272711189452008577645922249303295
num_cores = 100               # Number of parallel threads available
density = "1e20"              # Local DM energy density (in [Gev/cm^3])
method = "RK45"               # numerical integration method, to be passed to scipy.solve_ivp()

## Scenario to run
# NOTE: () means alias of previous listed item, [] indicates WIP/TODO scenarios.
scenario = "SIMPLE"           # one of: SINGLE, QCD, (AXION), SIMPLE, FULL, SU3, SU6, (SAMPLED), [REALS, NEUTRALS, COMPLEXES, CHARGED]

In [17]:
#%cd ..

In [18]:
#%load piaxiverse.py

In [19]:
#%conda init
#%conda activate piaxiverse

In [20]:
%%bash -s {density} {config_name} {seed} {num_cores} {verbosity} {use_mass_units} {method} {scenario} {save_output_files}

#module load anaconda
module load texlive
source /gpfs/runtime/opt/anaconda/latest/etc/profile.d/conda.sh
#conda activate piaxiverse

PIAXI_VERBOSITY=$5

if [[ "$PIAXI_VERBOSITY" -gt "6" ]]
then
    conda info
fi

if [[ "$6" = "False" ]]
then
    PIAXI_UNITS="--no-use_mass_units --no-use_natural_units"
else
    PIAXI_UNITS="--use_mass_units --use_natural_units"
fi

if [[ "$3" = "None" ]]
then
    PIAXI_SEED=""
else
    PIAXI_SEED="--seed $3"
fi

INPUT_ARG1="${8:-"0"}"
PIAXI_JOB_SUFFIX=""
PIAXI_FIT=""
if [[ "$INPUT_ARG1" = "0" ]]
then
    PIAXI_DQMC="1 1 1 0 0 0"
elif [[ "$INPUT_ARG1" = "SINGLE" ]]
then
    PIAXI_DQMC="0.5 0.5 0 0 0 0"
    PIAXI_JOB_SUFFIX="_single"
elif [[ "$INPUT_ARG1" = "QCD" ]] || [[ "$INPUT_ARG1" = "AXION" ]]
then
    PIAXI_DQMC="0.5 0.5 0 0 0 0"
    PIAXI_JOB_SUFFIX="_qcd"
    PIAXI_FIT="--fit_QCD"
elif [[ "$INPUT_ARG1" = "SIMPLE" ]]
then
    PIAXI_DQMC="1 1 1 0 0 0"
    PIAXI_JOB_SUFFIX="_simple"
elif [[ "$INPUT_ARG1" = "FULL" ]]
then
    PIAXI_DQMC="1 1 1 1 1 1"
    PIAXI_JOB_SUFFIX="_full"
elif [[ "$INPUT_ARG1" = "SU3" ]]
then
    PIAXI_DQMC="x x x 0 0 0"
    PIAXI_JOB_SUFFIX="_SU3"
elif [[ "$INPUT_ARG1" = "SAMPLED" ]] || [[ "$INPUT_ARG1" = "SU6" ]]
then
    PIAXI_DQMC="x x x x x x"
    PIAXI_JOB_SUFFIX="_SU6"
fi

PIAXI_SYS_NAME="$2${PIAXI_JOB_SUFFIX}"
PIAXI_N_CORES=$SLURM_JOB_CPUS_PER_NODE
PIAXI_N_NODES=$SLURM_JOB_NUM_NODES
PIAXI_COREMEM=$SLURM_MEM_PER_NODE
PIAXI_JOB_QOS=$SLURM_JOB_QOS

if [[ "True" = $9 ]]
then
    PIAXI_SAVE="--save_output_files --make_plots"
else
    PIAXI_SAVE="--no-save_output_files --no-make_plots"
fi

PIAXI_N_TIMES=300
PIAXI_T_MAX=30
PIAXI_N_KMODE=200
PIAXI_K_RES=0.1

# || For QCD axion case:
# || Expect resonance for cases where [sqrt(2*Rho)/m_a] >= [F_pi]
# ||   --> let m_a = 1e-6 eV
# ||   --> let g_a = 1e-10 GeV^-1
# || Then: expect critical threshold between rho ~ 1e17 GeV and 1e18 GeV

# Density [GeV] || For QCD axion case ~ (amp_a)^2*m_a / 2
PIAXI_DENSITY=$1
# F_pi [GeV] || For QCD axion case ~ 2/g_a
PIAXI_F="1e10"
# m_I [eV] || For QCD axion case ~ (m_a)^2 / F_pi
PIAXI_MASS="1e-80"

# Pi-Axiverse coupling constants: Lambda_3 and Lambda_4 [GeV]
PIAXI_L3="1e0"
PIAXI_L4="1e10"

PIAXI_EPS="1e0"
PIAXI_METHOD=$7

python piaxiverse.py $PIAXI_SEED $PIAXI_UNITS --int_method $PIAXI_METHOD --no-skip_existing --num_cores $PIAXI_N_CORES --job_qos $PIAXI_JOB_QOS --mem_per_core $PIAXI_COREMEM --num_samples 1 --verbosity $PIAXI_VERBOSITY --t $PIAXI_T_MAX --tN $PIAXI_N_TIMES --kN $PIAXI_N_KMODE --k_res $PIAXI_K_RES --m_scale $PIAXI_MASS --config_name $PIAXI_SYS_NAME --eps $PIAXI_EPS --L3 $PIAXI_L3 --L4 $PIAXI_L4 --rho $PIAXI_DENSITY --dqm_c $PIAXI_DQMC --F $PIAXI_F $PIAXI_FIT $PIAXI_SAVE



     active environment : piaxiverse
    active env location : /users/sloane1/.conda/envs/piaxiverse
            shell level : 1
       user config file : /users/sloane1/.condarc
 populated config files : /gpfs/runtime/opt/anaconda/2022.05/.condarc
          conda version : 4.12.0
    conda-build version : 3.21.8
         python version : 3.9.12.final.0
       virtual packages : __linux=5.14.0=0
                          __glibc=2.34=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /gpfs/runtime/opt/anaconda/2022.05  (read only)
      conda av data dir : /gpfs/runtime/opt/anaconda/2022.05/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : 

100%|██████████| 1991/1991 [00:28<00:00, 70.08it/s] 


  k=42.00  
  k=42.10  
  k=42.20  
  k=42.30  
  k=42.40  
  k=95.50  
  k=95.60  
  k=95.70  
  k=95.80  
  k=95.90  
  k=148.50  
  k=148.60  
  k=148.70  
  k=148.80  
  k=148.90  
  k=4.00  
  k=4.10  
  k=4.20  
  k=4.30  
  k=4.40  
  k=54.00  
  k=54.10  
  k=54.20  
  k=54.30  
  k=54.40  
  k=103.00  
  k=103.10  
  k=103.20  
  k=103.30  
  k=103.40  
  k=151.50  
  k=151.60  
  k=151.70  
  k=151.80  
  k=151.90  
  k=3.50  
  k=3.60  
  k=3.70  
  k=3.80  
  k=3.90  
  k=53.00  
  k=53.10  
  k=53.20  
  k=53.30  
  k=53.40  
  k=102.50  
  k=102.60  
  k=102.70  
  k=102.80  
  k=102.90  
  k=151.00  
  k=151.10  
  k=151.20  
  k=151.30  
  k=151.40  
  k=4.50  
  k=4.60  
  k=4.70  
  k=4.80  
  k=4.90  
  k=54.50  
  k=54.60  
  k=54.70  
  k=54.80  
  k=54.90  
  k=103.50  
  k=103.60  
  k=103.70  
  k=103.80  
  k=103.90  
  k=152.00  
  k=152.10  
  k=152.20  
  k=152.30  
  k=152.40  
  k=13.00  
  k=13.10  
  k=13.20  
  k=13.30  
  k=13.40  
  k=62.00  
  k=62.1